In [ ]:
import sys
import os
import subprocess

from pgpipe import four_gamete_pysam, vcf_to_ima, vcf_filter, vcf_calc, vcf_sampler, vcf_phase
from pgpipe.logging_module import initLogger
from pgpipe.informative_filter import filter_bed_regions
import pysam

print ("Imports complete")

In [ ]:
#Set up directories and filepaths, run on all restarts
work_dir='/home/jared/workspace/ppp/local_work/'
data_dir='/media/ccgg/ppp_sample_data/'
vcf_dir = work_dir+'great_ape_genome2/'
beagle_dir = '/home/jared/workspace/ppp/andrew/bin/'

main_vcf_name = data_dir+'recompress.vcf.gz'
filtered_vcf_pref = work_dir+'great_ape_genome/Pantrog_onlybiallelic_nomissing'
stat_file_pref = work_dir+'great_ape_genome2/fst.calc'
model_file = data_dir+'great_ape.model'
subsamp_bed_file = work_dir+'great_ape_genome2/5k_sample.bed'
logfile = '/home/jared/testpppj.log'


In [ ]:
#Define filenames for all loci, plus list of passed files
region_files = [vcf_dir+'Sample_Files/Sample_'+str(i)+'.vcf' for i in range(200)]
phased_files = [vcf_dir+'Phased/phased_'+str(i)+'.vcf' for i in range(200)]
fourg_files = [vcf_dir+'four_gamete/Sample_'+str(i)+'.vcf' for i in range(200)]
passed_files = []
initLogger(filename=logfile)

In [ ]:
%load_ext rpy2.ipython
from pgpipe import convert

In [ ]:
#Set up directory structure, only needs to be run once
if not os.path.exists(vcf_dir):
    os.makedirs(vcf_dir)
    os.makedirs(vcf_dir+'four_gamete/')
    os.makedirs(vcf_dir+'Sampled_nonmissing/')
    os.makedirs(vcf_dir+'Phased/')

In [ ]:
#Creates VCF filtered for no missing data and biallelic sites
vcf_filter.run(['--vcf', main_vcf_name, '--filter-max-missing', '1.0', '--model-file',model_file,
                '--model','2Pop', '--filter-min-alleles', '2', '--filter-max-alleles', '2', '--out-format', 
                'vcf.gz', '--out-prefix', filtered_vcf_pref, '--filter-exclude-chr', 'chrX', 'chrY', '--overwrite'])

pysam.tabix_index(filtered_vcf_pref+'.recode.vcf.gz',preset='vcf')
print("Filtering complete")

In [ ]:
#Calculates f_st statistics across genome
vcf_calc.run(['--vcf', filtered_vcf_pref + '.recode.vcf.gz', '--out-prefix', stat_file_pref, 
              '--calc-statistic', 'windowed-weir-fst', '--model', '2Pop', '--statistic-window-size', 
              '10000', '--statistic-window-step', '20000', '--model-file', model_file, '--overwrite'])
print("Stat calculation complete")

In [ ]:
#Selects subset of regions for fast sampling
filter_bed_regions(['--vcf',filtered_vcf_pref+'.recode.vcf.gz','--bed',stat_file_pref+'.windowed.weir.fst',
                    '--remove-indels','--minsites','4','--keep-full-line','--out',subsamp_bed_file,
                    '--randcount','5000'])
print("BED regions selected")


In [ ]:
#Uniformly sample regions for subset of 200 loci
vcf_sampler.run(['--vcf', filtered_vcf_pref + '.recode.vcf.gz', '--statistic-file', 
                 subsamp_bed_file, '--out-format', 'vcf', '--calc-statistic', 'windowed-weir-fst', 
                 '--sampling-scheme', 'uniform', '--uniform-bins', '5', '--out-dir', 
                 work_dir + 'great_ape_genome2/Sample_Files', '--overwrite'])

print("Sampling complete")


In [ ]:
#Phase locus
for i in range(200):
    vcf_phase.run(['--vcf',region_files[i],'--phase-algorithm','shapeit','--out',
                   phased_files[i],'--out-format','vcf','--overwrite'])
print ("Phasing done")

In [ ]:
#Subsample locus for four-gamete compatible interval, if no subregion returned, do not use VCF
for i in range(200):
    ret = four_gamete_pysam.sample_fourgametetest_intervals(['--vcfs', phased_files[i], '--out', 
                                                       fourg_files[i], '--4gcompat', '--reti', '--right', 
                                                       '--numinf', '2'])
    if ret is not None:
        passed_files.append(fourg_files[i])
print ("Four gamete regions selected for %d loci"%(len(passed_files)))

In [ ]:
#Create IMa input file
ima_args = ['--vcfs']
ima_args.extend(passed_files)
ima_args.extend(['--model-file', model_file, '--model','2Pop','--out', work_dir + 'ima_all_loci.ima.u'])

vcf_to_ima.vcf_to_ima(ima_args)
print ("IMa input created")

In [ ]:
#Admixture analysis, optional
from pgpipe import convert, admixture, graph_plotter
phased_string = ' '.join(phased_files)
loci_vcf = vcf_dir+'Phased/phased_merged.vcf.gz'
concatcall = subprocess.Popen('vcf-concat '+phased_string+ ' | bgzip -c > '+loci_vcf, shell=True,stdout=subprocess.PIPE)
temp_out, temp_err = concatcall.communicate()
convert.run(['--vcf',loci_vcf,'--out-format','binary-ped','--out-prefix',vcf_dir+'great_ape','--overwrite'])
admixture.run(['--file',vcf_dir+'great_ape.bed','--pop','2'])
graph_plotter.bar_plot(vcf_dir+'great_ape.2.Q')
print ("Plots created")